# apriori analysis

### after clustering,it will generate k classes,each class will have a list of user_ids inside that class,next step is to apply basket anaysis on each classification,to get the rules of each class

In [1]:
import pandas as pd
import numpy as np

aisle = pd.read_csv('../../DataSets/aisles.csv')
department = pd.read_csv('../../DataSets/departments.csv')
order_products_prior = pd.read_csv('../../DataSets/order_products__prior.csv')
order_products_train = pd.read_csv('../../DataSets/order_products__train.csv')
orders = pd.read_csv('../../DataSets/orders.csv')
products = pd.read_csv('../../DataSets/products.csv')

# fresh fruits:24
# fresh vegetables:83
# packaged vegetables fruits:122

In [2]:
import time
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth
from mlxtend.frequent_patterns import association_rules

def Apriori(user_product_part1):
    
    user_product_dic={}
    for item in user_product_part1.itertuples():
        if(user_product_dic.get(item[2])):
            user_product_dic[item[2]].append(item[3])
        else:
            user_product_dic[item[2]] = [] 
            user_product_dic[item[2]].append(item[3])
    transaction = []
    for item in user_product_dic:
        transaction.append(user_product_dic[item])
#     print('transaction',transaction)
    te = TransactionEncoder()
    te_ary = te.fit(transaction).transform(transaction)
#     print('te_ary',te_ary)
    df = pd.DataFrame(te_ary, columns=te.columns_)
    
    return df

In [3]:
def transform_associationRule_to_product(association_rule):
    
    relationship_rule = association_rule[['antecedents','consequents']]
    arr_relation=relationship_rule.values
    final_rules = []
    for item in arr_relation:
        pre_product_Arr = []
        after_product_Arr = []
        pre = item[0]
        after = item[1]
        for pre_item in pre:
            pre_product_Arr.append(products[products['product_id' ]== pre_item]['product_name'].values[0])
        for after_item in after:
            after_product_Arr.append(products[products['product_id' ]== after_item]['product_name'].values[0])
        tup = (pre_product_Arr,after_product_Arr)
        final_rules.append(tup)
    return final_rules

In [4]:
def transform_frequentItem_to_product(frequent_itemsets):
    arr_relation=frequent_itemsets.values
    final_arr = []
    for item in arr_relation:
        arr = []
        for it in item[1]:
            arr.append(products[products['product_id' ]== it]['product_name'].values[0])
        final_arr.append(arr)
    return final_arr

#  association items of all users (no any change)

In [5]:
order_user_id = orders[['order_id','user_id']]

order_products_id1 = order_products_train[['order_id','product_id']]
order_products_id2 = order_products_prior[['order_id','product_id']]
order_products_id = order_products_id1.append(order_products_id2, ignore_index=True)
# order_products_id = order_products_id2
order_products_id
user_product_id=order_user_id.merge(order_products_id,left_on='order_id', right_on='order_id')
user_product_id

,order_id,user_id,product_id
0,2539329,1,196
1,2539329,1,14084
2,2539329,1,12427
3,2539329,1,26088
4,2539329,1,26405
...,...,...,...
33819101,272231,206209,40603
33819102,272231,206209,15655
33819103,272231,206209,42606
33819104,272231,206209,37966


In [6]:
df = Apriori(user_product_id)
frequent_itemsets_all = apriori(df, min_support=0.1, use_colnames=True)
association_rule_all = association_rules(frequent_itemsets_all, metric="confidence", min_threshold=0.1)

In [7]:
transform_associationRule_to_product(association_rule_all)

[(['Bag of Organic Bananas'], ['Organic Strawberries']),
 (['Organic Strawberries'], ['Bag of Organic Bananas']),
 (['Bag of Organic Bananas'], ['Organic Baby Spinach']),
 (['Organic Baby Spinach'], ['Bag of Organic Bananas']),
 (['Bag of Organic Bananas'], ['Banana']),
 (['Banana'], ['Bag of Organic Bananas']),
 (['Bag of Organic Bananas'], ['Organic Hass Avocado']),
 (['Organic Hass Avocado'], ['Bag of Organic Bananas']),
 (['Banana'], ['Strawberries']),
 (['Strawberries'], ['Banana']),
 (['Organic Strawberries'], ['Organic Baby Spinach']),
 (['Organic Baby Spinach'], ['Organic Strawberries']),
 (['Organic Strawberries'], ['Banana']),
 (['Banana'], ['Organic Strawberries']),
 (['Limes'], ['Organic Strawberries']),
 (['Organic Strawberries'], ['Limes']),
 (['Organic Strawberries'], ['Organic Blueberries']),
 (['Organic Blueberries'], ['Organic Strawberries']),
 (['Organic Hass Avocado'], ['Organic Strawberries']),
 (['Organic Strawberries'], ['Organic Hass Avocado']),
 (['Organic Stra

In [8]:
s = frequent_itemsets_all.sort_values('support',ascending=False)[:20]
transform_frequentItem_to_product(s)


[['Banana'],
 ['Bag of Organic Bananas'],
 ['Organic Strawberries'],
 ['Organic Baby Spinach'],
 ['Large Lemon'],
 ['Limes'],
 ['Strawberries'],
 ['Organic Hass Avocado'],
 ['Organic Avocado'],
 ['Organic Blueberries'],
 ['Organic Garlic'],
 ['Organic Yellow Onion'],
 ['Organic Zucchini'],
 ['Organic Raspberries'],
 ['Bag of Organic Bananas', 'Organic Strawberries'],
 ['Yellow Onions'],
 ['Cucumber Kirby'],
 ['Organic Grape Tomatoes'],
 ['Organic Strawberries', 'Organic Baby Spinach'],
 ['Seedless Red Grapes']]

#  association items of all users (remove asiles : fresh fruits(24),fresh vegetables(83),packaged vegetables fruits(122))

### find the top 3 aisles [fresh fruits,fresh vegetables,packaged vegetables fruits]

In [9]:
aisle_ids = []
aisle_arr = aisle[aisle['aisle'].isin(['fresh fruits','fresh vegetables','packaged vegetables fruits'])].values
for item in aisle_arr:
    aisle_ids.append(item[0])

product_ids = []
product_arr = products[products['aisle_id'].isin(aisle_ids)].values
for item in product_arr:
    product_ids.append(item[0])
product_ids
aisle_ids

[24, 83, 123]

In [10]:
user_product_id = user_product_id[~user_product_id['product_id'].isin(product_ids)]

In [11]:
df = Apriori(user_product_id)
t1 = time.time()
frequent_itemsets = apriori(df, min_support=0.08, use_colnames=True)
t2 = time.time()
print('time t2 - t1',t2-t1)

time t2 - t1 29.57947611808777


In [12]:
association_rule = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.08)
# association_rule

In [13]:
transform_associationRule_to_product(association_rule)

[]

In [14]:
s = frequent_itemsets.sort_values('support',ascending=False)[:20]
transform_frequentItem_to_product(s)

[['Organic Cilantro'],
 ['Extra Virgin Olive Oil'],
 ['Organic Italian Parsley Bunch'],
 ['Blueberries'],
 ['Organic Whole Milk'],
 ['Bunched Cilantro'],
 ['Original Hummus'],
 ['Raspberries'],
 ['Grated Parmesan'],
 ['Sparkling Water Grapefruit'],
 ['Boneless Skinless Chicken Breasts'],
 ['Organic Half & Half'],
 ['Half & Half'],
 ['100% Whole Wheat Bread']]

# after removing frequent asiles and apply association rule on each user classification 

In [15]:
user_classification = pd.read_csv('../customer_classificationV2.csv')
user_classification = user_classification[['user_id','label']]
user0 = user_classification[user_classification['label']==0]
user1 = user_classification[user_classification['label']==1]
user2 = user_classification[user_classification['label']==2]
user3 = user_classification[user_classification['label']==3]
user4 = user_classification[user_classification['label']==4]

In [16]:
def apply_apriori_fun(dataset,minSup,minCon):
    df = Apriori(dataset)
    t1 = time.time()
    frequent_itemsets0 = apriori(df, min_support=minSup, use_colnames=True)
    t2 = time.time()
    print('time t2 - t1',t2-t1)

    association_rule0 = association_rules(frequent_itemsets0, metric="confidence", min_threshold=minCon)
    # association_rule0
    
    s = frequent_itemsets0.sort_values('support',ascending=False)[:20]
    print('top frequent items')
    frequent_items = transform_frequentItem_to_product(s)
    for item in frequent_items:
        print(item)
    
#     return association_rule0[['antecedents','consequents']]
    return transform_associationRule_to_product(association_rule0)

#  association items of clustering 0

In [17]:
user_product_part0 = user_product_id[user_product_id['user_id'].isin(user0['user_id']) ]

### top rules and top items of train data (70%)

In [18]:
x0=apply_apriori_fun(user_product_part0[:int(len(user_product_part0)*0.7)],0.08,0.1)
x0

time t2 - t1 2.6710798740386963
top frequent items
['Organic Cilantro']
['Organic Italian Parsley Bunch']
['Extra Virgin Olive Oil']
['Organic Whole Milk']
['Blueberries']
['Bunched Cilantro']
['Grated Parmesan']
['Original Hummus']
['Unsalted Butter']
['Half & Half']
['Organic Black Beans']
['Organic Basil']
['Boneless Skinless Chicken Breasts']
['Feta Cheese Crumbles']
['Organic Garbanzo Beans']
['Organic Half & Half']
['Organic Thyme']
['Organic Cilantro', 'Organic Italian Parsley Bunch']
['Organic Coconut Milk']
['Organic Sour Cream']


[(['Organic Cilantro'], ['Organic Whole Milk']),
 (['Organic Whole Milk'], ['Organic Cilantro']),
 (['Organic Whole Milk'], ['Organic Italian Parsley Bunch']),
 (['Organic Italian Parsley Bunch'], ['Organic Whole Milk']),
 (['Bunched Cilantro'], ['Organic Cilantro']),
 (['Organic Cilantro'], ['Bunched Cilantro']),
 (['Bunched Cilantro'], ['Organic Italian Parsley Bunch']),
 (['Organic Italian Parsley Bunch'], ['Bunched Cilantro']),
 (['Extra Virgin Olive Oil'], ['Organic Cilantro']),
 (['Organic Cilantro'], ['Extra Virgin Olive Oil']),
 (['Extra Virgin Olive Oil'], ['Organic Italian Parsley Bunch']),
 (['Organic Italian Parsley Bunch'], ['Extra Virgin Olive Oil']),
 (['Grated Parmesan'], ['Organic Cilantro']),
 (['Organic Cilantro'], ['Grated Parmesan']),
 (['Organic Cilantro'], ['Organic Italian Parsley Bunch']),
 (['Organic Italian Parsley Bunch'], ['Organic Cilantro']),
 (['Grated Parmesan'], ['Organic Italian Parsley Bunch']),
 (['Organic Italian Parsley Bunch'], ['Grated Parmesan'

### top rules and top items of test data (30%)

In [19]:
apply_apriori_fun(user_product_part0[-int(len(user_product_part0)*0.3):],0.08,0.1)

time t2 - t1 0.8488929271697998
top frequent items
['Extra Virgin Olive Oil']
['Organic Cilantro']
['Organic Italian Parsley Bunch']
['Organic Whole Milk']
['Blueberries']
['Bunched Cilantro']
['Grated Parmesan']
['Original Hummus']
['Unsalted Butter']
['Organic Black Beans']
['Half & Half']
['Organic Half & Half']
['Boneless Skinless Chicken Breasts']
['Organic Basil']
['Feta Cheese Crumbles']
['Organic Coconut Milk']
['Organic Garbanzo Beans']
['Organic Cilantro', 'Organic Italian Parsley Bunch']
['Organic Whole String Cheese']
['Raspberries']


[(['Organic Whole Milk'], ['Organic Cilantro']),
 (['Organic Cilantro'], ['Organic Whole Milk']),
 (['Organic Whole Milk'], ['Organic Italian Parsley Bunch']),
 (['Organic Italian Parsley Bunch'], ['Organic Whole Milk']),
 (['Bunched Cilantro'], ['Organic Cilantro']),
 (['Organic Cilantro'], ['Bunched Cilantro']),
 (['Bunched Cilantro'], ['Organic Italian Parsley Bunch']),
 (['Organic Italian Parsley Bunch'], ['Bunched Cilantro']),
 (['Extra Virgin Olive Oil'], ['Organic Cilantro']),
 (['Organic Cilantro'], ['Extra Virgin Olive Oil']),
 (['Extra Virgin Olive Oil'], ['Organic Italian Parsley Bunch']),
 (['Organic Italian Parsley Bunch'], ['Extra Virgin Olive Oil']),
 (['Organic Cilantro'], ['Organic Italian Parsley Bunch']),
 (['Organic Italian Parsley Bunch'], ['Organic Cilantro']),
 (['Grated Parmesan'], ['Organic Italian Parsley Bunch']),
 (['Organic Italian Parsley Bunch'], ['Grated Parmesan'])]

#  association items of clustering 1

In [20]:
user_product_part1 = user_product_id[user_product_id['user_id'].isin(user1['user_id']) ]

### top rules and top items of train data (70%)

In [21]:
x1=apply_apriori_fun(user_product_part1[:int(len(user_product_part1)*0.7)],0.13,0.1)
x1

time t2 - t1 0.10761189460754395
top frequent items
['Organic Whole Milk']
['Baby Food Stage 2 Blueberry Pear & Purple Carrot']
['Spinach Peas & Pear Stage 2 Baby Food']
['Organic Whole String Cheese']
['Grated Parmesan']
['Organic Italian Parsley Bunch']
['Extra Virgin Olive Oil']
['Broccoli & Apple Stage 2 Baby Food']
['Original Hummus']
['Organic Cilantro']
['Stage 1 Apples Sweet Potatoes Pumpkin & Blueberries Organic Pureed Baby Food']
['Organic Mixed Berry Yogurt & Fruit Snack']
['Blueberries']
['Organic Pears, Peas and Broccoli Puree Stage 1']
['Organic Black Beans']
['Happy Baby Spinach, Mango, and Pear Baby Food']
['Peach,  Apricot & Banana Stage 2 Baby Food']
['Organic Strawberry Yogurt & Fruit Snack']
['Organic Stage 4 Green Bean Pear & Pea Baby Food']
['Feta Cheese Crumbles']


[(['Baby Food Stage 2 Pumpkin Banana'],
  ['Broccoli & Apple Stage 2 Baby Food']),
 (['Broccoli & Apple Stage 2 Baby Food'],
  ['Baby Food Stage 2 Pumpkin Banana']),
 (['Broccoli & Apple Stage 2 Baby Food'], ['Organic Whole Milk']),
 (['Organic Whole Milk'], ['Broccoli & Apple Stage 2 Baby Food']),
 (['Broccoli & Apple Stage 2 Baby Food'],
  ['Spinach Peas & Pear Stage 2 Baby Food']),
 (['Spinach Peas & Pear Stage 2 Baby Food'],
  ['Broccoli & Apple Stage 2 Baby Food']),
 (['Baby Food Stage 2 Blueberry Pear & Purple Carrot'],
  ['Broccoli & Apple Stage 2 Baby Food']),
 (['Broccoli & Apple Stage 2 Baby Food'],
  ['Baby Food Stage 2 Blueberry Pear & Purple Carrot']),
 (['Apple and Carrot Stage 2 Baby Food'],
  ['Broccoli & Apple Stage 2 Baby Food']),
 (['Broccoli & Apple Stage 2 Baby Food'],
  ['Apple and Carrot Stage 2 Baby Food']),
 (['Peach,  Apricot & Banana Stage 2 Baby Food'],
  ['Spinach Peas & Pear Stage 2 Baby Food']),
 (['Spinach Peas & Pear Stage 2 Baby Food'],
  ['Peach,  Apr

### top rules and top items of test data (30%)

In [22]:
apply_apriori_fun(user_product_part1[-int(len(user_product_part1)*0.3):],0.13,0.1)

time t2 - t1 0.04680228233337402
top frequent items
['Organic Whole Milk']
['Baby Food Stage 2 Blueberry Pear & Purple Carrot']
['Spinach Peas & Pear Stage 2 Baby Food']
['Broccoli & Apple Stage 2 Baby Food']
['Grated Parmesan']
['Organic Whole String Cheese']
['Extra Virgin Olive Oil']
['Organic Italian Parsley Bunch']
['Organic Pears, Peas and Broccoli Puree Stage 1']
['Stage 1 Apples Sweet Potatoes Pumpkin & Blueberries Organic Pureed Baby Food']
['Peach,  Apricot & Banana Stage 2 Baby Food']
['Happy Baby Spinach, Mango, and Pear Baby Food']
['Organic Cilantro']
['Sweeet Potato, Apple, Carrot & Cinnamon Organic Superfoods']
['Baby Food Stage 2 Pumpkin Banana']
['Blueberries']
['Baby Food Stage 2 Blueberry Pear & Purple Carrot', 'Spinach Peas & Pear Stage 2 Baby Food']
['Baby Food Stage 2 Blueberry Pear & Purple Carrot', 'Broccoli & Apple Stage 2 Baby Food']
['Organic Stage 4 Spinach Mango & Pear Baby Food']
['Organic Mixed Berry Yogurt & Fruit Snack']


[(['Peach,  Apricot & Banana Stage 2 Baby Food'],
  ['Broccoli & Apple Stage 2 Baby Food']),
 (['Broccoli & Apple Stage 2 Baby Food'],
  ['Peach,  Apricot & Banana Stage 2 Baby Food']),
 (['Baby Food Stage 2 Pumpkin Banana'],
  ['Broccoli & Apple Stage 2 Baby Food']),
 (['Broccoli & Apple Stage 2 Baby Food'],
  ['Baby Food Stage 2 Pumpkin Banana']),
 (['Happy Baby Spinach, Mango, and Pear Baby Food'],
  ['Broccoli & Apple Stage 2 Baby Food']),
 (['Broccoli & Apple Stage 2 Baby Food'],
  ['Happy Baby Spinach, Mango, and Pear Baby Food']),
 (['Broccoli & Apple Stage 2 Baby Food'], ['Organic Whole Milk']),
 (['Organic Whole Milk'], ['Broccoli & Apple Stage 2 Baby Food']),
 (['Stage 2 Sweet Potato Corn & Apple Baby Food'],
  ['Broccoli & Apple Stage 2 Baby Food']),
 (['Broccoli & Apple Stage 2 Baby Food'],
  ['Stage 2 Sweet Potato Corn & Apple Baby Food']),
 (['Broccoli & Apple Stage 2 Baby Food'],
  ['Spinach Peas & Pear Stage 2 Baby Food']),
 (['Spinach Peas & Pear Stage 2 Baby Food'],
 

#  association items of clustering 2

In [36]:
user_product_part2 = user_product_id[user_product_id['user_id'].isin(user2['user_id']) ]

### top rules and top items of train data (70%)

In [41]:
x2=apply_apriori_fun(user_product_part2[:int(len(user_product_part2)*0.7)],0.08,0.1)
x2

time t2 - t1 1.4773061275482178
top frequent items
['Extra Virgin Olive Oil']
['Organic Cilantro']
['Blueberries']
['Organic Italian Parsley Bunch']
['Original Hummus']
['Organic Whole String Cheese']
['Uncured Genoa Salami']
['Grated Parmesan']
['Organic Whole Milk']
['Organic Black Beans']
['Honey Nut Cheerios']
['Half & Half']
['Organic Half & Half']
['Bunched Cilantro']
['Frozen Organic Wild Blueberries']
['Organic Ketchup']
['100% Whole Wheat Bread']
['Organic Unsweetened Almond Milk']
['Organic Whole Strawberries']
['Feta Cheese Crumbles']


[(['Extra Virgin Olive Oil'], ['Blueberries']),
 (['Blueberries'], ['Extra Virgin Olive Oil']),
 (['Organic Whole Strawberries'], ['Frozen Organic Wild Blueberries']),
 (['Frozen Organic Wild Blueberries'], ['Organic Whole Strawberries']),
 (['Original Hummus'], ['Organic Whole String Cheese']),
 (['Organic Whole String Cheese'], ['Original Hummus']),
 (['Extra Virgin Olive Oil'], ['Organic Black Beans']),
 (['Organic Black Beans'], ['Extra Virgin Olive Oil']),
 (['Organic Black Beans'], ['Organic Cilantro']),
 (['Organic Cilantro'], ['Organic Black Beans']),
 (['Organic Black Beans'], ['Organic Italian Parsley Bunch']),
 (['Organic Italian Parsley Bunch'], ['Organic Black Beans']),
 (['Bunched Cilantro'], ['Organic Cilantro']),
 (['Organic Cilantro'], ['Bunched Cilantro']),
 (['Original Hummus'], ['Extra Virgin Olive Oil']),
 (['Extra Virgin Olive Oil'], ['Original Hummus']),
 (['Original Hummus'], ['Organic Italian Parsley Bunch']),
 (['Organic Italian Parsley Bunch'], ['Original Hum

### top rules and top items of test data (30%)

In [42]:
apply_apriori_fun(user_product_part2[-int(len(user_product_part2)*0.3):],0.08,0.1)

time t2 - t1 0.6664149761199951
top frequent items
['Extra Virgin Olive Oil']
['Organic Cilantro']
['Organic Italian Parsley Bunch']
['Blueberries']
['Original Hummus']
['Organic Black Beans']
['Grated Parmesan']
['Organic Whole String Cheese']
['Organic Whole Milk']
['Uncured Genoa Salami']
['Half & Half']
['Sparkling Water Grapefruit']
['Organic Half & Half']
['Bunched Cilantro']
['100% Whole Wheat Bread']
['Frozen Organic Wild Blueberries']
['Shredded Parmesan']
['Organic Unsweetened Almond Milk']
['Lightly Salted Baked Snap Pea Crisps']
['Organic Whole Strawberries']


[(['Grated Parmesan'], ['Shredded Parmesan']),
 (['Shredded Parmesan'], ['Grated Parmesan']),
 (['Organic Whole Strawberries'], ['Frozen Organic Wild Blueberries']),
 (['Frozen Organic Wild Blueberries'], ['Organic Whole Strawberries']),
 (['Organic Whole String Cheese'], ['Organic Whole Milk']),
 (['Organic Whole Milk'], ['Organic Whole String Cheese']),
 (['Original Hummus'], ['Organic Whole String Cheese']),
 (['Organic Whole String Cheese'], ['Original Hummus']),
 (['Extra Virgin Olive Oil'], ['Organic Whole String Cheese']),
 (['Organic Whole String Cheese'], ['Extra Virgin Olive Oil']),
 (['Organic Whole String Cheese'], ['Organic Cilantro']),
 (['Organic Cilantro'], ['Organic Whole String Cheese']),
 (['Organic Whole String Cheese'], ['Grated Parmesan']),
 (['Grated Parmesan'], ['Organic Whole String Cheese']),
 (['Organic Whole String Cheese'], ['Organic Italian Parsley Bunch']),
 (['Organic Italian Parsley Bunch'], ['Organic Whole String Cheese']),
 (['Organic Coconut Milk'], 

#  association items of clustering 3

In [26]:
user_product_part3 = user_product_id[user_product_id['user_id'].isin(user3['user_id']) ]

### top rules and top items of train data (70%)

In [27]:
x3=apply_apriori_fun(user_product_part3[:int(len(user_product_part3)*0.7)],0.2,0.2)
x3

time t2 - t1 0.010338783264160156
top frequent items
['Hass Avocados']
['Clementines']
['Raspberries']
['Packaged Grape Tomatoes']
['Seedless Cucumbers']
['Clementines', 'Hass Avocados']
['Raspberries', 'Hass Avocados']
['Rainbow Bell Peppers']
['Clementines', 'Raspberries']
['Baby Cucumbers']
['Packaged Grape Tomatoes', 'Hass Avocados']
['Seedless Cucumbers', 'Hass Avocados']
['Packaged Grape Tomatoes', 'Clementines']
['Green Seedless Grapes']
['Organic Tortilla Chips']
['Rainbow Bell Peppers', 'Hass Avocados']
['Packaged Grape Tomatoes', 'Raspberries']
['Blueberries']
['Clementines', 'Raspberries', 'Hass Avocados']
['Raspberries', 'Seedless Cucumbers']


[(['Clementines'], ['Soda']),
 (['Soda'], ['Clementines']),
 (['Packaged Grape Tomatoes'], ['Clementines']),
 (['Clementines'], ['Packaged Grape Tomatoes']),
 (['Clementines'], ['Baby Cucumbers']),
 (['Baby Cucumbers'], ['Clementines']),
 (['Clementines'], ['Blueberries']),
 (['Blueberries'], ['Clementines']),
 (['Clementines'], ['Organic Simply Naked Pita Chips']),
 (['Organic Simply Naked Pita Chips'], ['Clementines']),
 (['Clementines'], ['Hass Avocados']),
 (['Hass Avocados'], ['Clementines']),
 (['Clementines'], ['Green Seedless Grapes']),
 (['Green Seedless Grapes'], ['Clementines']),
 (['Clementines'], ['Rainbow Bell Peppers']),
 (['Rainbow Bell Peppers'], ['Clementines']),
 (['Clementines'], ['Seedless Cucumbers']),
 (['Seedless Cucumbers'], ['Clementines']),
 (['Clementines'], ['Organic Tortilla Chips']),
 (['Organic Tortilla Chips'], ['Clementines']),
 (['Clementines'], ['Raspberries']),
 (['Raspberries'], ['Clementines']),
 (['Packaged Grape Tomatoes'], ['Baby Cucumbers']),


### top rules and top items of test data (30%)

In [28]:
apply_apriori_fun(user_product_part3[-int(len(user_product_part3)*0.3):],0.2,0.2)

time t2 - t1 0.008121013641357422
top frequent items
['Hass Avocados']
['Clementines']
['Raspberries']
['Packaged Grape Tomatoes']
['Clementines', 'Hass Avocados']
['Baby Cucumbers']
['Seedless Cucumbers']
['Raspberries', 'Hass Avocados']
['Raspberries', 'Clementines']
['Rainbow Bell Peppers']
['Packaged Grape Tomatoes', 'Hass Avocados']
['Blueberries']
['Packaged Grape Tomatoes', 'Clementines']
['Seedless Cucumbers', 'Hass Avocados']
['Baby Cucumbers', 'Hass Avocados']
['Packaged Grape Tomatoes', 'Raspberries']
['Clementines', 'Baby Cucumbers']
['Rainbow Bell Peppers', 'Hass Avocados']
['Broccoli Florettes']
['Sweet Kale Salad Mix']


[(['Clementines'], ['Soda']),
 (['Soda'], ['Clementines']),
 (['Broccoli Florettes'], ['Hass Avocados']),
 (['Hass Avocados'], ['Broccoli Florettes']),
 (['Raspberries'], ['Broccoli Florettes']),
 (['Broccoli Florettes'], ['Raspberries']),
 (['Packaged Grape Tomatoes'], ['Clementines']),
 (['Clementines'], ['Packaged Grape Tomatoes']),
 (['Clementines'], ['Baby Cucumbers']),
 (['Baby Cucumbers'], ['Clementines']),
 (['Clementines'], ['Blueberries']),
 (['Blueberries'], ['Clementines']),
 (['Clementines'], ['Organic Simply Naked Pita Chips']),
 (['Organic Simply Naked Pita Chips'], ['Clementines']),
 (['Clementines'], ['Hass Avocados']),
 (['Hass Avocados'], ['Clementines']),
 (['Clementines'], ['Green Seedless Grapes']),
 (['Green Seedless Grapes'], ['Clementines']),
 (['Clementines'], ['Rainbow Bell Peppers']),
 (['Rainbow Bell Peppers'], ['Clementines']),
 (['Clementines'], ['Seedless Cucumbers']),
 (['Seedless Cucumbers'], ['Clementines']),
 (['Clementines'], ['Organic Tortilla Chip

#  association items of clustering 4

In [29]:
user_product_part4 = user_product_id[user_product_id['user_id'].isin(user4['user_id']) ]

In [30]:
x4=apply_apriori_fun(user_product_part4[:int(len(user_product_part4)*0.7)],0.1,0.1)
x4

time t2 - t1 0.2102210521697998
top frequent items
['Sparkling Water Grapefruit']
['Lime Sparkling Water']
['Extra Virgin Olive Oil']
['Sparkling Lemon Water']
['Sparkling Water Grapefruit', 'Lime Sparkling Water']
['Sparkling Natural Mineral Water']
['Organic Italian Parsley Bunch']
['Organic Cilantro']
['Organic Half & Half']
['Spring Water']
['Half & Half']
['Blueberries']
['Original Hummus']
['Peach Pear Flavored Sparkling Water']
['Sparkling Water Grapefruit', 'Sparkling Lemon Water']
['Sparkling Lemon Water', 'Lime Sparkling Water']
['Organic Whole Milk']
['Sparkling Water Berry']
['100% Recycled Paper Towels']
['Grated Parmesan']


[(['Sparkling Water Grapefruit'], ['Pure Sparkling Water']),
 (['Pure Sparkling Water'], ['Sparkling Water Grapefruit']),
 (['Sparkling Lemon Water'], ['Sparkling Water Berry']),
 (['Sparkling Water Berry'], ['Sparkling Lemon Water']),
 (['Peach Pear Flavored Sparkling Water'], ['Sparkling Water Berry']),
 (['Sparkling Water Berry'], ['Peach Pear Flavored Sparkling Water']),
 (['Lime Sparkling Water'], ['Sparkling Water Berry']),
 (['Sparkling Water Berry'], ['Lime Sparkling Water']),
 (['Sparkling Water Grapefruit'], ['Sparkling Water Berry']),
 (['Sparkling Water Berry'], ['Sparkling Water Grapefruit']),
 (['Peach Pear Flavored Sparkling Water'], ['Sparkling Lemon Water']),
 (['Sparkling Lemon Water'], ['Peach Pear Flavored Sparkling Water']),
 (['Sparkling Lemon Water'], ['Lime Sparkling Water']),
 (['Lime Sparkling Water'], ['Sparkling Lemon Water']),
 (['Sparkling Water Grapefruit'], ['Sparkling Lemon Water']),
 (['Sparkling Lemon Water'], ['Sparkling Water Grapefruit']),
 (['Peac

In [31]:
apply_apriori_fun(user_product_part4[-int(len(user_product_part4)*0.3):],0.1,0.1)

time t2 - t1 0.08635115623474121
top frequent items
['Sparkling Water Grapefruit']
['Lime Sparkling Water']
['Extra Virgin Olive Oil']
['Sparkling Lemon Water']
['Organic Half & Half']
['Sparkling Water Grapefruit', 'Lime Sparkling Water']
['Half & Half']
['Peach Pear Flavored Sparkling Water']
['Original Hummus']
['Organic Cilantro']
['Organic Italian Parsley Bunch']
['Blueberries']
['Sparkling Natural Mineral Water']
['Sparkling Water Grapefruit', 'Sparkling Lemon Water']
['Sparkling Water Berry']
['Sparkling Water Grapefruit', 'Peach Pear Flavored Sparkling Water']
['Pure Sparkling Water']
['Sparkling Lemon Water', 'Lime Sparkling Water']
['Organic Whole Milk']
['Spring Water']


[(['Kiwi Sandia Sparkling Water'], ['Blackberry Cucumber Sparkling Water']),
 (['Blackberry Cucumber Sparkling Water'], ['Kiwi Sandia Sparkling Water']),
 (['Sparkling Water Grapefruit'], ['Kiwi Sandia Sparkling Water']),
 (['Kiwi Sandia Sparkling Water'], ['Sparkling Water Grapefruit']),
 (['Pure Sparkling Water'], ['Lime Sparkling Water']),
 (['Lime Sparkling Water'], ['Pure Sparkling Water']),
 (['Sparkling Water Grapefruit'], ['Pure Sparkling Water']),
 (['Pure Sparkling Water'], ['Sparkling Water Grapefruit']),
 (['Sparkling Lemon Water'], ['Sparkling Water Berry']),
 (['Sparkling Water Berry'], ['Sparkling Lemon Water']),
 (['Peach Pear Flavored Sparkling Water'], ['Sparkling Water Berry']),
 (['Sparkling Water Berry'], ['Peach Pear Flavored Sparkling Water']),
 (['Lime Sparkling Water'], ['Sparkling Water Berry']),
 (['Sparkling Water Berry'], ['Lime Sparkling Water']),
 (['Sparkling Water Grapefruit'], ['Sparkling Water Berry']),
 (['Sparkling Water Berry'], ['Sparkling Water G

In [43]:
output_rules0 = pd.DataFrame(x0)
output_rules0.columns=['first','second']
output_rules0['label']=0


output_rules1 = pd.DataFrame(x1)
output_rules1.columns=['first','second']
output_rules1['label']=1

output_rules2 = pd.DataFrame(x2)
output_rules2.columns=['first','second']
output_rules2['label']=2

output_rules3 = pd.DataFrame(x3)
output_rules3.columns=['first','second']
output_rules3['label']=3

output_rules4 = pd.DataFrame(x4)
output_rules4.columns=['first','second']
output_rules4['label']=4

output_rules = pd.concat([output_rules0,output_rules1,output_rules2,output_rules3,output_rules4],ignore_index=True)
output_rules

,first,second,label
0,[Organic Cilantro],[Organic Whole Milk],0
1,[Organic Whole Milk],[Organic Cilantro],0
2,[Organic Whole Milk],[Organic Italian Parsley Bunch],0
3,[Organic Italian Parsley Bunch],[Organic Whole Milk],0
4,[Bunched Cilantro],[Organic Cilantro],0
...,...,...,...
333,"[Sparkling Water Grapefruit, Lime Sparkling Wa...",[Peach Pear Flavored Sparkling Water],4
334,"[Peach Pear Flavored Sparkling Water, Lime Spa...",[Sparkling Water Grapefruit],4
335,[Sparkling Water Grapefruit],"[Peach Pear Flavored Sparkling Water, Lime Spa...",4
336,[Peach Pear Flavored Sparkling Water],"[Sparkling Water Grapefruit, Lime Sparkling Wa...",4


In [44]:
output_rules.to_csv('./output_Aprioir.csv')

In [34]:
# output=[]
# for item in output_rules.values:
#     arr = []
#     arr.append(list(item[0]))
#     arr.append(list(item[1]))
#     arr.append(item[2])
#     output.append(arr)
# output

In [35]:
# x = pd.DataFrame(output)
# x.columns=['first','second','label']
# x.to_csv('./output_Aprioir.csv')
# x